# Introduction to Quantum Computing Project
## Traveling Salesman Problem

**Students**
* Matheus Silva Melo de Oliveira
* Nívea de Abreu Dantas Lima

# The Traveling Salesman Problem

The Traveling Salesman Problem (TSP) is a well-known and notoriously complex problem in the field of combinatorial optimization. It is often used to exemplify the challenges and computational difficulties associated with solving optimization problems. TSP can be succinctly described as follows:

**Problem Statement:**

*Given a list of cities and the distances between each pair of cities, the objective of the TSP is to find the shortest possible route that visits each city exactly once and returns to the starting city.*

## Complexity

TSP belongs to a class of problems known as NP-hard (Nondeterministic Polynomial-time hard), which means that it is computationally intractable to find an optimal solution in polynomial time as the problem size increases. The complexity of TSP arises from several factors:

1. **Exponential Growth**: As the number of cities (n) increases, the number of possible routes to consider grows factorially, making the problem's solution space explode. There are n! (n factorial) possible permutations of cities to visit, which quickly becomes unmanageable for large values of n.

2. **Non-Polynomial Time**: TSP belongs to the class of problems that do not have known algorithms to solve them in polynomial time. This means that as the problem size grows, the time required to find an optimal solution grows exponentially.

3. **Combinatorial Nature**: TSP is a combinatorial optimization problem, which requires evaluating all possible combinations to find the optimal route. This makes it computationally demanding because it demands searching through an immense solution space.

## Computational Difficulty

The computational difficulty of TSP has several real-world implications:

1. **Resource-Intensive**: Solving TSP for a large number of cities can take a significant amount of time and computational resources, making it impractical for many real-world applications.

2. **Heuristic Approaches**: To address the computational difficulty, heuristic and approximation algorithms are often used to find near-optimal solutions. These algorithms sacrifice guaranteed optimality for computational efficiency.

3. **Applications**: TSP has applications in various fields, such as logistics, transportation, and circuit design. The computational difficulty of solving TSP has real-world implications in these domains, as finding the most efficient routes can significantly impact costs and resources.

In conclusion, the Traveling Salesman Problem is a classic example of a computationally challenging problem due to its exponential growth in solution space, non-polynomial time complexity, and combinatorial nature. While there are ways to tackle TSP, finding the optimal solution for large instances remains a daunting computational task.

## Force Brute Approach
We can iterate all permutations of paths and select the best one. 

Although it always return the optimal solution it is too expensive in computational terms.

In [1]:
import itertools

def calculate_total_distance(path, distances):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distances[path[i]][path[i + 1]]
    total_distance += distances[path[-1]][path[0]]
    return total_distance

def brute_force_tsp(distances):
    num_cities = len(distances)
    cities = list(range(num_cities))
    shortest_path = None
    min_distance = float('inf')

    for perm in itertools.permutations(cities):
        total_distance = calculate_total_distance(perm, distances)
        if total_distance < min_distance:
            min_distance = total_distance
            shortest_path = perm

    return shortest_path, min_distance

distances = [
    [0, 29, 20, 21],
    [29, 0, 15, 18],
    [20, 15, 0, 28],
    [21, 18, 28, 0]
]

shortest_path, min_distance = brute_force_tsp(distances)
print("Shortest Path:", shortest_path)
print("Minimum Distance:", min_distance)

Shortest Path: (0, 2, 1, 3)
Minimum Distance: 74


## Heuristic Approach:

We can use one of the properties of the problem and use it to achieve good solutions in better time complexity. One possible on is selecting the nearest nodes around the current node.

In [2]:
import numpy as np

def nearest_neighbor_tsp(distances):
    num_cities = len(distances)
    unvisited_cities = list(range(num_cities))
    path = [0] 
    unvisited_cities.remove(0)

    while unvisited_cities:
        current_city = path[-1]
        nearest_city = min(unvisited_cities, key=lambda city: distances[current_city][city])
        path.append(nearest_city)
        unvisited_cities.remove(nearest_city)

    path.append(path[0])

    total_distance = sum(distances[path[i]][path[i + 1]] for i in range(num_cities))

    return path, total_distance

distances = np.array([
    [0, 29, 20, 21],
    [29, 0, 15, 18],
    [20, 15, 0, 28],
    [21, 18, 28, 0]
])

shortest_path, min_distance = nearest_neighbor_tsp(distances)
print("Shortest Path:", shortest_path)
print("Minimum Distance:", min_distance)


Shortest Path: [0, 2, 1, 3, 0]
Minimum Distance: 74


## Quantum Computing Approach

As we've seen, in classical computing, there isn't a optimal solution in polynomial time, we have to choose one to be privileged. 

However, using **Quantum parallelism** could optimize the solution and give optimial solutions in almost polynomial time.

## Why Quantum Computing can solve some NP-hard problems

Quantum computing and the concept of parallelism hold great promise in addressing NP (nondeterministic polynomial) problems, which encompass a wide range of computationally challenging issues. The core advantage of quantum computing lies in its ability to manipulate quantum bits or qubits, which can exist in multiple states simultaneously due to superposition. This inherent parallelism allows quantum computers to explore numerous potential solutions at the same time, significantly reducing the time required to find optimal answers.

Furthermore, qubits can be entangled, meaning the state of one qubit depends on the state of another. This entanglement enables the exploration of complex relationships and dependencies within NP problems, providing a powerful tool for efficient problem-solving.

One well-known algorithm that leverages quantum parallelism is Grover's algorithm, which offers a quadratic speedup for unstructured search problems, such as finding an item in an unsorted database.

In addition to superposition and entanglement, quantum annealing, a specialized form of quantum computing, can be employed to optimize solutions for specific NP problems. Quantum annealers exploit quantum tunneling to navigate complex solution spaces more efficiently.

The advantages of parallelism extend beyond quantum computing. Classical parallel computing, which involves breaking a problem into smaller subtasks that can be processed simultaneously, also plays a significant role in addressing NP problems. By distributing computational load across multiple processors or cores, parallel computing can reduce the time required for complex calculations.

In some cases, hybrid models that combine quantum and classical computing techniques are employed. These models allow quantum processors to handle the most computationally intensive portions of NP problems while classical computers manage the remaining aspects.

## Using Quantum Computing to solve TSP problem

### Defining the Quantum Oracle: Phase Estimation

Since the input for the problem is a matrix containing the respective distances from the current node to a chosen node within the node space presented in the problem. In this way, as we want to perform a search over all possible paths (permutations) and obtain all possible distances for each of these combinations, it is interesting to use a quantum phase estimation oracle that can represent all the phases related to the eigenvectors of the underlying unitary operations.

Initially, we need to create a unitary operator that can represent, in terms of complex numbers, each distance from node i to node j.

$$b_{ij} = e^{i \cdot c_{ij}}$$

Thus, for each city on the diagonal of the unitary matrix, one can construct a unitary operator U such that:

$$u_{ij}^i = b_{ij}$$

Constructing each of these "path" operators, we can see that the final operation U will be the result of tensor products.

$$U = U^{(1)} \otimes \ldots \otimes U^{(n)}$$

Once we obtain the eigenvectors of the operator U, we see that multiplying U by its own eigenvectors generates the total length of the Hamiltonian cycle. This cycle refers to a circuit or path that passes through all vertices of a graph exactly once and returns to the initial vertex. This configuration allows us to use a phase estimation algorithm to obtain the duration of any Hamiltonian cycle. We obtain the phases in the form of binary output from the phase estimation algorithm, so we can easily run the quantum algorithm to find the minimum path among all generated permutations.


In [1]:
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *

# qiskit-ibmq-provider has been deprecated.
# Please see the Migration Guides in https://ibm.biz/provider_migration_guide for more detail.
from qiskit_ibm_runtime import QiskitRuntimeService, Sampler, Estimator, Session, Options

# Loading your IBM Quantum account(s)
service = QiskitRuntimeService(channel="ibm_quantum")

# Invoke a primitive. For more details see https://qiskit.org/documentation/partners/qiskit_ibm_runtime/tutorials.html
# result = Sampler("ibmq_qasm_simulator").run(circuits).result()

## References
* https://medium.com/@vasilybokov/traveling-salesman-problem-with-quantum-optimization-solutions-and-perspectives-59137f3241cd

* https://nferrazzo.com/static/media/UNIGOU.21b0d123.pdf

